Cargamos la librería que nos permite acceder al dataset

In [ ]:
# soporte para cargar dataset de https://www.openml.org/
!pip install openml
import openml

Accedemos al dataset

In [ ]:
import pandas as pd

# indicamos cual dataset queremos utilizar, en este caso el nro. 24
dataset = openml.datasets.get_dataset(24)

# separamos las información almacenada en el dataset
X, y, categorical_indicator, attribute_names = dataset.get_data(
    dataset_format='dataframe',
    target=dataset.default_target_attribute
)

#  concatenamos la información relevante en un único DataFrame 
df = pd.concat([X, y], axis=1)
df


,cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,NaN,s,s,o,o,p,o,o,p,b,c,l,e
8120,x,s,n,f,n,a,c,b,y,e,NaN,s,s,o,o,p,n,o,p,b,v,l,e
8121,f,s,n,f,n,a,c,b,n,e,NaN,s,s,o,o,p,o,o,p,b,c,l,e
8122,k,y,n,f,y,f,c,n,b,t,NaN,s,k,w,w,p,w,o,e,w,v,l,p


# a) Describir el dataset seleccionado. 
*Este dataset se trata de hongos, describe sus caracteristicas fisicas. En este dataset se clasifican en venenosos o comestibles. En este dataset cuenta con 8124 ejemplares y contiene 22 caracteristicas ademas agrega una columna mas que la usa como un target que contiene si es comestible o venenoso.*

# b) Definir cuál es el objetivo de aplicar la técnica de Reglas de Asociación en el dataset elegido. Explicar qué tipo de patrones se esperan encontrar.
*El objetivo de aplicar la tecnica de reglas de asociacion sera a encontrar relaciones entre los hongos mas relevantes de este conjunto de datos.* 


# c) Definir qué pre‐procesamiento se realizará a los datos. Justificar. Aplicarlo. 
*Para el pre-procesamiento de los datos se utilizara la funcion de get_dummies la cual nos proporciona* **Pandas** *para poder generar la representacion de one hot encoded, eso lo conseguimos en el df_dummies. Use para pre-procesar lo explicado con anterioridad ya que es mas facil poder visualizarlo. Tambien elegi que los datos frecuentes tengan un soporte mayor a 0.3 ya que es un dataset bastante grande, intente con 0.2 pero no me deja.* 


In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori

df_dummies = pd.get_dummies(df, columns=df.columns)
#df_dummies Ya esta en formato one-hot
frequent_itemsets = apriori(df_dummies, min_support=0.3, use_colnames=True)
frequent_itemsets['lenght'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets


,support,itemsets,lenght
0,0.387986,(cap-shape_f),1
1,0.450025,(cap-shape_x),1
2,0.314623,(cap-surface_s),1
3,0.399311,(cap-surface_y),1
4,0.584441,(bruises%3F_f),1
...,...,...,...
2728,0.311177,"(ring-number_o, veil-color_w, ring-type_p, vei...",8
2729,0.303299,"(gill-size_b, veil-color_w, bruises%3F_t, veil...",9
2730,0.317085,"(gill-size_b, ring-number_o, veil-color_w, rin...",9
2731,0.303299,"(gill-size_b, veil-color_w, ring-type_p, bruis...",9


# e) Obtener las reglas de asociación utilizando el dataset elegido. Definir parámetros del algoritmo. Justificar. 

*A continuacion defino las reglas de asociacion obtenidas, esto lo voy a definir con una confianza de 0.9 esto con el fin de filtrar aquellas que no sean suficientemente seguras para los datos.*

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.9)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(cap-shape_f),(gill-attachment_f),0.387986,0.974151,0.381339,0.982868,1.008949,0.003382,1.508835
1,(cap-shape_f),(veil-type_p),0.387986,1.000000,0.387986,1.000000,1.000000,0.000000,inf
2,(cap-shape_f),(veil-color_w),0.387986,0.975382,0.381832,0.984137,1.008976,0.003397,1.551945
3,(cap-shape_f),(ring-number_o),0.387986,0.921713,0.371738,0.958122,1.039501,0.014126,1.869388
4,(cap-shape_x),(gill-attachment_f),0.450025,0.974151,0.443378,0.985230,1.011373,0.004986,1.750096
...,...,...,...,...,...,...,...,...,...
24402,"(ring-type_p, bruises%3F_t, gill-attachment_f,...","(ring-number_o, veil-color_w, veil-type_p, sta...",0.350566,0.484490,0.316100,0.901685,1.861101,0.146254,5.243469
24403,"(ring-type_p, bruises%3F_t, stalk-surface-abov...","(ring-number_o, veil-color_w, veil-type_p, gil...",0.341704,0.772033,0.316100,0.925072,1.198228,0.052294,3.042476
24404,"(ring-type_p, bruises%3F_t, gill-spacing_c, st...","(ring-number_o, veil-color_w, veil-type_p, gil...",0.337765,0.559330,0.316100,0.935860,1.673179,0.127178,6.870440
24405,"(bruises%3F_t, ring-number_o, stalk-surface-be...","(veil-color_w, ring-type_p, veil-type_p, gill-...",0.337765,0.385032,0.316100,0.935860,2.430603,0.186050,9.587910


# f) Determinar qué post‐procesamiento se realizará sobre las reglas obtenidas. Aplicarlo. 

*El post-procesado inical va ser obtener aquellas reglas que me determinaran a un hongo dentro de la clase venenosa. Analizando todas las reglas obtenidas, pueden observarse dos mas generales. Para esas dos reglas mas generales aplique dos filtros.*

In [ ]:
print("Se imprimen las reglas que satisfacen las condiciones \n")

#Basicamente necesito una regla que me lleve a deducir si un hongo es venenoso. Por ende hizo esto:
print("Regla:")
print(rules[(rules['consequents'] == {'class_p'})])

#Por la tabla de arriba voy a filtrar buscando el par bruises%3F_f y gill-spacing_c
print("Filtro 1:")
print(rules[(rules['antecedents'].apply(lambda x: "bruises%3F_f" in x)) & (rules['antecedents'].apply(lambda x: "gill-spacing_c" in x)) & (rules['lift'] > 1.01) & (rules['consequents'] == {'class_p'})])

#Por la tabla de arriba voy a filtrar buscando el par bruises%3F_f o population_v que es la segunda que aparece en la primera regla.
print("Filtro 2:")
print(rules[(rules['antecedents'].apply(lambda x: "bruises%3F_f" in x)) & ((rules['antecedents'].apply(lambda x: "gill-spacing_c" in x)) | (rules['antecedents'].apply(lambda x: "population_v" in x)))  & (rules['lift'] > 1.01) & (rules['consequents'] == {'class_p'})])


Se imprimen las reglas que satisfacen las condiciones 

Regla:
                                             antecedents  ... conviction
206                       (gill-spacing_c, bruises%3F_f)  ...   5.209145
238                         (bruises%3F_f, population_v)  ...   6.522603
1084   (gill-spacing_c, bruises%3F_f, gill-attachment_f)  ...  10.780281
1160     (population_v, bruises%3F_f, gill-attachment_f)  ...  11.326309
1171         (gill-spacing_c, veil-type_p, bruises%3F_f)  ...   5.209145
1182        (veil-color_w, gill-spacing_c, bruises%3F_f)  ...  10.838552
1192       (gill-spacing_c, bruises%3F_f, ring-number_o)  ...   5.377037
1195        (gill-spacing_c, bruises%3F_f, population_v)  ...   8.090221
1260           (veil-type_p, population_v, bruises%3F_f)  ...   6.522603
1277          (veil-color_w, bruises%3F_f, population_v)  ...  11.326309
1281         (bruises%3F_f, population_v, ring-number_o)  ...   6.522603
3713   (gill-spacing_c, veil-type_p, gill-attachment_...  ...

# g) Analizar las reglas de asociación obtenidas.

*Del analisis del ejercicio anterior vemos que con las condificiones de filtro que he establecido, se puede deducir que todas las reglas que tienen a "Bruises%3F_f" y "Gill-spacing_c", o "Bruises%3F_f" y "Population_v", y todos estos son hongos de clase venenoso.*

In [ ]:
rules [ (rules['antecedents'].apply(lambda x: len(x)) < 3) & (rules['lift'] > 1.01) & (rules['consequents'] == {'class_p'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
206,"(gill-spacing_c, bruises%3F_f)",(class_p),0.435746,0.482029,0.392418,0.900565,1.868281,0.182376,5.209145
238,"(bruises%3F_f, population_v)",(class_p),0.334810,0.482029,0.308223,0.920588,1.909821,0.146834,6.522603


# h) Mostrar un ejemplo de cómo los resultados obtenidos pueden ser usados en el futuro.

*Estos resultados pueden ser utilizados en el futuro como una herramienta o un buscador de hongos venenosos, por ejemplo se podria implementar una web para buscar hongos por imagenes y que por su categoria te diga si es venenoso o no, se me ocurre una red neuronal.*